In [1]:
import numpy as np
import zlib
from PIL import Image
import base64
import cv2
import io
import json

def base64_2_mask(s):
    z = zlib.decompress(base64.b64decode(s))
    n = np.fromstring(z, np.uint8)
    mask = cv2.imdecode(n, cv2.IMREAD_UNCHANGED)[:, :, 3].astype(bool)
    return mask

PROJECT_PATH = '/home/asya/studying/Проекты/Распознавание серийных номеров/object detection/'
GOOD_IMAGES_ANN = PROJECT_PATH + 'appropriate/ann/'
BAD_IMAGES_ANN = PROJECT_PATH + 'bad/ann/'

def create_txt_file(labels_path: str, json_path: str, image_filename: str):
    with open(json_path) as f:
        file_content = f.read()
        annotation = json.loads(file_content)
    img_height = annotation['size']['height']
    img_width = annotation['size']['width']
    image_name = image_filename.replace('.png', '').replace('.jpg', '')
    with open(labels_path + image_name + '.txt', 'w') as f:
        for i, obj in enumerate(annotation['objects']):
            left_corner_x, left_corner_y = [int(s) for s in annotation['objects'][i]['bitmap']['origin']]
            bbox_height, bbox_width = base64_2_mask(annotation['objects'][i]['bitmap']['data']).shape
            x_center = left_corner_x + bbox_width // 2
            y_center = left_corner_y + bbox_height // 2
            output_arr = [0, x_center / img_width, y_center / img_height, bbox_width / img_width, bbox_height / img_height]
            output_str = ' '.join([str(s) for s in output_arr])
            f.write(output_str + '\n')

In [2]:
create_txt_file(PROJECT_PATH + 'appropriate/labels/', GOOD_IMAGES_ANN + 'IMG_1749.png.json', 'IMG_1749.png')

/tmp/ipykernel_10360/1085451713.py:11: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  n = np.fromstring(z, np.uint8)
